Завантаження файлу з ПК.

Автоматичне визначення розділювача та кодування.

Додавання колонки RowID.

Аналіз конфліктів: Артикул продукта має різні Продукти на одному і тому ж Штрихкоді.

Вивід результату + збереження у CSV для скачування.

In [4]:
import pandas as pd
from google.colab import files

# 1. Завантаження файлу з ПК
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 2. Функція для "розумного" читання CSV
def smart_read_csv(filepath):
    separators = [";", ",", "\t"]
    encodings = ["utf-8", "cp1251"]
    best_df = None
    best_cols = 0

    for sep in separators:
        for enc in encodings:
            try:
                df = pd.read_csv(filepath, sep=sep, encoding=enc)
                if df.shape[1] > best_cols:  # беремо той варіант, де більше колонок
                    best_cols = df.shape[1]
                    best_df = df
                    best_sep = sep
                    best_enc = enc
            except:
                pass

    if best_df is not None:
        print(f"✅ Файл прочитано успішно! Розділювач: '{best_sep}', кодування: {best_enc}, колонок: {best_cols}")
        return best_df
    else:
        raise ValueError("❌ Не вдалося прочитати CSV. Спробуйте вручну вказати sep та encoding.")

# 3. Читаємо CSV
df = smart_read_csv(filename)

# 4. Додаємо порядковий номер
df.insert(0, "RowID", range(1, len(df) + 1))

# 5. Пошук конфліктів: один Артикул → різні Продукти на одному Штрихкоді
conflicts = (
    df.groupby(["Штрихкод ячейки", "Артикул продукта"])["Продукт"]
    .nunique()
    .reset_index()
)

conflicts = conflicts[conflicts["Продукт"] > 1]  # залишаємо лише конфлікти

# 6. Витягуємо детальні рядки з оригінальної таблиці
conflict_rows = df.merge(
    conflicts[["Штрихкод ячейки", "Артикул продукта"]],
    on=["Штрихкод ячейки", "Артикул продукта"],
    how="inner"
)

print("🔎 Знайдені конфлікти (Артикул продукта → різні Продукти на одному Штрихкоді):")
display(conflict_rows)

# 7. Зберігаємо результат у CSV
output_filename = "conflicts_result.csv"
conflict_rows.to_csv(output_filename, index=False, encoding="utf-8-sig")

files.download(output_filename)


Saving ДублікатиЦСКиїв - 04.09.2025.csv to ДублікатиЦСКиїв - 04.09.2025 (2).csv
✅ Файл прочитано успішно! Розділювач: ';', кодування: utf-8, колонок: 3
🔎 Знайдені конфлікти (Артикул продукта → різні Продукти на одному Штрихкоді):


,RowID,Штрихкод ячейки,Артикул продукта,Продукт
0,681,01-03-C-16-C-01,8859471,00-00021632
1,682,01-03-C-16-C-01,8859471,10000116961
2,1335,01-03-A-15-A-01,500086329,10006337217
3,1336,01-03-A-15-A-01,500086329,10006424681
4,1628,01-03-C-15-C-01,5801754016,10000096316
5,1629,01-03-C-15-C-01,5801754016,10000184724
6,5470,01-13-A-01-C-01,OT__W10X38/FRONT/JD,10000214659
7,5471,01-13-A-01-C-01,OT__W10X38/FRONT/JD,10000214658
8,5472,01-13-A-01-C-01,OT__W10X54/BACK/JD6,10000214661
9,5473,01-13-A-01-C-01,OT__W10X54/BACK/JD6,10000214660


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>